In [1]:
!pip install pygame
# ばくだんくん

import pygame
import random

##### Global Variables #####
SCREEN_WIDTH = 452   #画面のサイズ
SCREEN_HEIGHT = 352  #
COLOR_BLACK = (0,0,0)  # 黒
COLOR_RED =(255,0,0)   # 赤
DX = [-1, 0, 1, 0]   # 向きごとの移動分
DY = [0, -1, 0, 1]   #
stageNumber = 1  # 現在のステージ番号
bombKazu = 1   # 置くことのできる爆弾の数。初期数は同時には1つまで設置可能
bombLength = 1   # 爆弾の威力。初期は1つ先のマスまで
mapAll = open("./mapdata.txt", "r").readlines()
#print("mapAll", mapAll)

##### Class #####
class MyChar(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        
        self.alive = True
        self.muki = 0
        self.jyoutai = 0
        
        self.images = []
        for i in range(4):
            self.images.append(pygame.image.load("./img/my" + str(i) + ".png"))
        self.image = self.images[self.muki]
        self.rect = self.image.get_rect()
        colorkey = self.image.get_at((0,0))
        print(colorkey)
        self.image.set_colorkey(colorkey)

    def changeImage(self):
        self.image = self.images[self.muki]
        
        
class TekiChar(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        
        self.alive = True
        self.muki = 0
        self.jyoutai = 0
        
        self.images = []
        for i in range(3):
            self.images.append(pygame.image.load("./img/teki" + str(i) + ".png"))
        self.image = self.images[self.muki]
        self.rect = self.image.get_rect()

    def update(self):
        if self.jyoutai <= 0:
            self.muki = random.randrange(4)
            self.jyoutai = 32
            xnext = (self.rect.x - 100) // 32 + DX[self.muki]
            ynext = (self.rect.y      ) // 32 + DY[self.muki]
            if maps[ynext][xnext] != "0":
                self.jyoutai = 0
        else:
            self.image = self.images[(self.rect.x + self.rect.y) % 3]
            self.rect.x += DX[self.muki]
            self.rect.y += DY[self.muki]
            self.jyoutai -= 1

class Bomb(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)

        self.jyoutai = 100

        self.images = []
        for i in range(4):
            self.images.append(pygame.image.load("./img/bomb" + str(i) + ".png"))
        self.image = self.images[0]
        self.rect = self.image.get_rect()

    def setYX(self, y, x):
        self.rect.x = x * 32 + 100
        self.rect.y = y * 32

    def update(self):
        self.jyoutai -= 1
        if self.jyoutai > 70:
            pass
        elif self.jyoutai > 40:
            self.image = self.images[1]
        elif self.jyoutai > 10:
            self.image = self.images[2]
        else:
            self.image = self.images[3]
            if self.jyoutai > 0:
                # 爆発の描画のみ
                bakuhaCheck(False, self.rect.y // 32, (self.rect.x - 100) // 32)
            else:
                # 実際に壁や敵や自分を破壊する
                bakuhaCheck(True, self.rect.y // 32, (self.rect.x - 100) // 32)
                x = (self.rect.x - 100) // 32
                y =  self.rect.y        // 32
                maps[y][x] = "0"
                bombGroup.remove(self)
    

        
##### SubRoutine #####
def bakuhaCheck(hakaiFlag, y, x):
    global flg
    wx = x - 1 ; wy = y     ; count = 0 ; flg = 0
    while (wx > 0) and (count < bombLength) and (flg == 0):
        bakuha(hakaiFlag, wy, wx)
        wx -= 1
        count += 1
    wx = x     ; wy = y - 1 ; count = 0 ; flg = 0
    while (wy > 0) and (count < bombLength) and (flg == 0):
        bakuha(hakaiFlag, wy, wx)
        wy -= 1
        count += 1
    wx = x + 1 ; wy = y     ; count = 0 ; flg = 0
    while (wx < 11) and (count < bombLength) and (flg == 0):
        bakuha(hakaiFlag, wy, wx)
        wx += 1
        count += 1
    wx = x     ; wy = y + 1 ; count = -1 ; flg = 0
    while (wy >= 0) and (count < bombLength) and (flg == 0):
        bakuha(hakaiFlag, wy, wx)
        wy += 1
        count += 1
    
def bakuha(hakaiFlag, by, bx):
    global flg
    if maps[by][bx] == "0": # 通路だったら
        screen.blit(imgExplosion, (bx * 32 + 100, by * 32))
    #elif maps[by][bx] == "1": # 扉（出現ずみ）
    #    pass
    elif maps[by][bx] == "6": # 他の爆弾だったら
        for bomb in bombGroup.sprites():
            if bomb.jyoutai > 10:
                if (bomb.rect.x - 100) // 32 == bx and bomb.rect.y // 32 == by :
                    bomb.jyoutai = 10
    elif maps[by][bx] == "7": # 扉（壁に隠れている）
        if hakaiFlag == 1:
            maps[by][bx] = "1"
            flg = 1
    elif maps[by][bx] == "8": # 壊れない壁
        flg = 1
    elif maps[by][bx] == "9": # 壊れる壁
        if hakaiFlag == True:
            maps[by][bx] = "0"
            r = random.randrange(100)
            if r < 4:
                maps[by][bx] = "2"
            elif r < 8:
                maps[by][bx] = "3"
            elif r < 16:
                maps[by][bx] = "4"
        flg = 1
    for tekiChar in tekiCharGroup.sprites():
        if (tekiChar.rect.x - 100) // 32 == bx and tekiChar.rect.y // 32 == by:
            tekiCharGroup.remove(tekiChar)
    
def gamenByouga():
    screen.fill(COLOR_BLACK)
    for y in range(11):
        for x in range(11):
            #pygame.draw.rect(screen, (255,255,255), (x * 32, y * 32, 32, 32))
            if maps[y][x] == "8": # 壊れない壁
                screen.blit(imgBlock, (x * 32 + 100, y * 32))
            elif maps[y][x] == "9" or maps[y][x] == "7": #壊れる壁
                screen.blit(imgWall, (x * 32 + 100, y * 32))
            elif maps[y][x] == "1": # 扉（出現ずみ）
                screen.blit(imgDoor, (x * 32 + 100, y * 32))
            elif maps[y][x] == "2": # 爆発範囲増加アイテム
                screen.blit(imgItemFire, (x * 32 + 100, y * 32))
            elif maps[y][x] == "3": # 爆弾所持数増加アイテム
                screen.blit(imgItemBomb, (x * 32 + 100, y * 32))
            elif maps[y][x] == "4": # 爆発範囲減少アイテム
                screen.blit(imgItemWater, (x * 32 + 100, y * 32))
            else: # 通路
                screen.blit(imgFloor, (x * 32 + 100, y * 32))

def mapsSetup():
    global tekiCharGroup
    tekiCharGroup = pygame.sprite.Group()
    global maps
    maps = []
    for y in range(11):
        rec = mapAll[(stageNumber - 1) * 12 + y]
        rec = rec[:-1]
        maps.append(list(rec))
        for x in range(11):
            if maps[y][x] == "M":
                myChar.rect.x = 32 * x + 100
                myChar.rect.y = 32 * y
                maps[y][x] = "0"
            elif maps[y][x] == "E":
                tekiChar = TekiChar()
                tekiChar.rect.x = 32 * x + 100
                tekiChar.rect.y = 32 * y
                tekiCharGroup.add(tekiChar)
                maps[y][x] = "0"
            elif maps[y][x] == "G":
                maps[y][x] = "7"
    #print(maps)
            
#######################################################
##### GAME START
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH,SCREEN_HEIGHT))
gameclock = pygame.time.Clock()

### Charcter Generate
myCharGroup = pygame.sprite.Group()
myChar = MyChar()
myCharGroup.add(myChar)

##### Char Tile & Item & ExplosionMark #####
imgFloor = pygame.image.load("./img/tileFloor.png").convert()
imgBlock = pygame.image.load("./img/tileBlock.png").convert()
imgWall = pygame.image.load("./img/tileWall.png").convert()
imgDoor = pygame.image.load("./img/tileDoor.png").convert()

imgItemFire = pygame.image.load("./img/itemFire.png").convert()
imgItemBomb = pygame.image.load("./img/itemBomb.png").convert()
imgItemWater = pygame.image.load("./img/itemWater.png").convert()

imgExplosion = pygame.image.load("./img/bomb3.png").convert()


stageNumber = 1
##### Stage Start
while stageNumber <= 3:
    mapsSetup()
    bombGroup = pygame.sprite.Group()

    mainLoopEndFlag = False

    while mainLoopEndFlag == False:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                endflag = 1

        keypress = pygame.key.get_pressed()
        x = (myChar.rect.x - 100) // 32
        y = myChar.rect.y // 32
        if myChar.jyoutai == 0: # ちょうどマス目上にいますよ
            # ステージクリアの判定
            if maps[y][x] == "1" and len(tekiCharGroup.sprites()) == 0:
                mainLoopEndFlag = True
            
            # アイテムゲットのチェック
            if maps[y][x] == "2": # 爆発範囲アップ！
                bombLength += 1
                maps[y][x] = "0"
            elif maps[y][x] == "3": # 爆弾所持数アップ！
                bombKazu += 1
                maps[y][x] = "0"
            elif maps[y][x] == "4": # 爆発範囲ダウン
                bombLength -= 1
                maps[y][x] = "0"

            # 移動のチェック    
            if (keypress[pygame.K_LEFT]):
                myChar.muki = 0
                nextcell = maps[y + DY[myChar.muki]][x + DX[myChar.muki]]
                if int(nextcell) <= 4:
                    myChar.jyoutai = 8
            elif (keypress[pygame.K_UP]):
                myChar.muki = 1
                nextcell = maps[y + DY[myChar.muki]][x + DX[myChar.muki]]
                if int(nextcell) <= 4:
                    myChar.jyoutai = 8
            elif (keypress[pygame.K_RIGHT]):
                myChar.muki = 2
                nextcell = maps[y + DY[myChar.muki]][x + DX[myChar.muki]]
                if int(nextcell) <= 4:
                    myChar.jyoutai = 8
            elif (keypress[pygame.K_DOWN]):
                myChar.muki = 3
                nextcell = maps[y + DY[myChar.muki]][x + DX[myChar.muki]]
                if int(nextcell) <= 4:
                    myChar.jyoutai = 8
            # 爆弾設置のチェック
            elif (keypress[pygame.K_SPACE]):
                if (maps[y][x] == "0"):  #通路にしか置けない
                    if len(bombGroup.sprites()) < bombKazu:
                        bomb = Bomb()
                        bomb.setYX(y, x)
                        bombGroup.add(bomb)
                        maps[y][x] = "6"
            myChar.changeImage()
        else:
            myChar.rect.x += (DX[myChar.muki] * 4)
            myChar.rect.y += (DY[myChar.muki] * 4)
            myChar.jyoutai -= 1

        gamenByouga() # 背景となるマップや残り時間などを描画
        tekiCharGroup.update() # 敵の移動処理
        tekiCharGroup.draw(screen) #敵の描画
        bombGroup.update() # 爆弾の状態を進行させる
        bombGroup.draw(screen) # 爆弾の描画
        myCharGroup.draw(screen) #自機の描画

        gameclock.tick(30)
        pygame.display.flip()
        
        
    stageNumber += 1

pygame.quit()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
(1, 0, 0, 255)
